## PyTorch Workflow

Pytorch end-to-end workflow.
1. Prepare and load data
2. Build model
3. Fitting the model to data (training)
4. Make predictions and evaulating model
5. Saving and loading model

### 1. Prepare and load data

In [2]:
import torch
from torch import nn # nn contains all the building blocks for model in pytorch
import matplotlib.pyplot as plt

In [8]:
weight = 0.8
bias = 0.3
start = 0
end = 1

X = torch.arange(start, end, step=0.2).unsqueeze(dim=1)
y = weight * X + bias

#### Splitting data into training and test sets

In [14]:
train_split = int(0.8 * len(X))
Xtrain, ytrain = X[:train_split], y[:train_split]
Xtest, ytest = X[train_split:], y[train_split:]

### 2. Build a model

In [13]:
from torch import nn

# create linear regression model class
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.weights = nn.Parameter(torch.randn(1,
                                                requires_grad=True,
                                                dtype=torch.float))
        self.bias = nn.Parameter(torch.randn(1, requires_grad=True,
                                             dtype=torch.float))
    # Forward method to define the computation in the model    
    def forward(self, x:torch.Tensor)->torch.tensor:
        return self.weights * x + self.bias

4